In [52]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
import data_preprocessing
import importlib
importlib.reload(data_preprocessing)
import cross_validation
import TEST
importlib.reload(TEST)
importlib.reload(cross_validation)
import implemented_functions
importlib.reload(implemented_functions)



<module 'implemented_functions' from '/Users/christophersoriano/Library/Mobile Documents/com~apple~CloudDocs/Mac/Epfl/Master/MA3/Machine Learning/ML_project1/project1/implemented_functions.py'>

In [12]:
np.random.seed(42)

In [13]:
#Load Data
x_train_raw, x_test_raw,y_train_raw, train_ids, test_ids = load_csv_data('dataset')

In [14]:
print(x_train_raw.shape)
print(x_test_raw.shape)


(328135, 321)
(109379, 321)


In [15]:
#Reformat y_train to have 0's instead of -1's
y_train_original = y_train_raw.copy()
y_train_raw[y_train_raw == -1] = 0

In [16]:
#replace NaN by a float to treat NaN as a categorical feature
x_test_raw = np.nan_to_num(x_test_raw, nan = -10.0)
x_test_raw.shape

(109379, 321)

### One Hot Encode

In [17]:
data_annoted = data_preprocessing.read_annotated_csv('dataset/data_anotated.csv')

In [21]:
x_train, y_train, x_test = data_preprocessing.preprocess_data2(x_train_raw[:50000,:], y_train_raw[:50000], x_test_raw[:50000,:], data_annoted)


In [ ]:
x_test

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:
"""x_train_filtered = data_preprocessing.remove_useless(x_train_raw, data_annoted)
x_test_filtered = data_preprocessing.remove_useless(x_test_raw, data_annoted)"""

'x_train_filtered = data_preprocessing.remove_useless(x_train_raw, data_annoted)\nx_test_filtered = data_preprocessing.remove_useless(x_test_raw, data_annoted)'

In [ ]:
"""data_train = data_preprocessing.clean_data(x_train_filtered[:100000,:], data_annoted)
data_test = data_preprocessing.clean_data(x_test_filtered[:100000,:], data_annoted)"""

'data_train = data_preprocessing.clean_data(x_train_filtered[:100000,:], data_annoted)\ndata_test = data_preprocessing.clean_data(x_test_filtered[:100000,:], data_annoted)'

In [22]:
np.savetxt('x_train_preprocessed.csv', x_train, delimiter=',')
np.savetxt('y_train_preprocessed.csv', y_train, delimiter=',')
np.savetxt('x_test_preprocessed.csv', x_test, delimiter=',')

In [53]:
x_train = np.loadtxt('x_train_preprocessed.csv', delimiter=',')
x_test = np.loadtxt('x_test_preprocessed.csv', delimiter=',')
y_train = np.loadtxt('y_train_preprocessed.csv', delimiter=',')

In [54]:

initial_w = np.zeros(x_train.shape[1])
max_iters = 50

x_test = x_train[7000:,:]
y_test = y_train[7000:]
x_train = x_train[:7000,:]
y_train = y_train[:7000]


w_least_squares, loss_ls = implemented_functions.least_squares(y_train, x_train)
print(f"Least Squares: Loss = {loss_ls}")

best_mse_coef = 0
best_mse_loss = np.inf
best_mse_weights = None
for gamma in [0.01, 0.1, 0.5, 1]:
    w_sgd, loss_sgd = implemented_functions.mean_squared_error_sgd(y_train,  x_train, initial_w, max_iters, gamma)
    if loss_sgd[-1] < best_mse_loss:
        best_mse_loss = loss_sgd[-1]
        best_mse_coef = gamma
        best_mse_weights = w_sgd
    print(f"Mean Squared Error SGD (gamma={gamma}): Loss = {loss_sgd}")

best_log_lasso_coef = 0
best_log_lasso_loss = np.inf
best_log_lasso_weights = None
for gamma in [0.01, 0.1, 0.5]:
    for lambda_ in [0.1, 0.01]:
        w_log_lasso, loss_log_lasso = implemented_functions.reg_logistic_lasso_subgradient(y_train, x_train, lambda_, initial_w, max_iters, gamma)
        if loss_log_lasso < best_log_lasso_loss:
            best_log_lasso_loss = loss_log_lasso
            best_log_lasso_weights = w_log_lasso
        print(f"Regularized Lasso Logistic Regression (gamma={gamma}, lambda={lambda_}): Loss = {loss_log_lasso}")
        
best_ridge_coef = 0
best_ridge_loss = np.inf
best_ridge_weights = None
for ridge_coef in [0.001, 0.01, 0.1, 1, 10]:
    w_ridge, loss_ridge = implemented_functions.ridge_regression(y_train, x_train, ridge_coef)
    if loss_ridge < best_ridge_loss:
        best_ridge_loss = loss_ridge
        best_ridge_coef = ridge_coef
        best_ridge_weights = w_ridge
    print(f"Ridge Regression (alpha={ridge_coef}): Loss = {loss_ridge}")

best_log_gamma = 0
best_log_loss = np.inf
best_log_weights = None
for gamma in [0.01, 0.1, 0.5, 1]:
    w_logistic, loss_logistic = implemented_functions.logistic_regression(y_train, x_train, initial_w, max_iters, gamma)
    if loss_logistic < best_log_loss:
        best_log_loss = loss_logistic
        best_log_gamma = gamma
        best_log_weights = w_logistic
    print(f"Logistic Regression: Loss = {loss_logistic}")

best_log_reg_coef = 0
best_log_reg_loss = np.inf
best_log_reg_weights = None
for gamma in [0.01, 0.1, 0.5]:
    for reg_coef in [0.001, 0.01, 0.1, 1]:
        w_reg_logistic, loss_reg_logistic = implemented_functions.reg_logistic_regression(y_train, x_train, reg_coef, initial_w, max_iters, gamma)
        if loss_reg_logistic < best_log_reg_loss:
            best_log_reg_loss = loss_reg_logistic
            best_log_reg_coef = reg_coef
            best_log_reg_weights = w_reg_logistic
        print(f"Regularized Logistic Regression (alpha={reg_coef}, gamma={gamma}): Loss = {loss_reg_logistic}")



Least Squares: Loss = 0.02937233535573916
Mean Squared Error SGD (gamma=0.01): Loss = [0.043428571428571434, 0.043428571428571434, 0.09046186662122753, 0.054125165634448066, 0.04229686038454425, 0.04072663567268243, 0.04088354581770833, 0.041993604620169996, 0.0427165517232729, 0.04379471077892396, 0.043324995087639595, 0.04434855794550884, 0.04376844345222379, 0.04263595874806054, 0.04307930548847357, 0.04215898039417133, 0.04282388797614339, 0.04148889137845025, 0.047983136937909086, 0.046373539241470245, 0.04389427699455997, 0.042515888285590886, 0.10736085908804023, 0.047473174719232306, 0.03910224560871096, 0.03833094834263337, 0.03857217604386013, 0.0383722054902626, 0.039311660111795856, 0.038718794489773785, 0.038518564963564296, 0.03997660506248597, 0.03989963397011998, 0.03980521839749573, 0.044462524233832346, 0.049553049980584005, 0.04608804415504674, 0.047150460705441745, 0.04211324078800699, 0.04332384474804474, 0.045247887436550484, 0.04043690153642787, 0.039547753895686

In [13]:
np.max(x_train@best_log_weights)

3.07318898540649

In [55]:

import implemented_functions

def test_methods(x_test, y_test, w_sgd, w_least_squares, best_ridge_weights, best_log_weights, best_log_reg_weights, threshold=0.5):
    accuracy, y_pred = implemented_functions.compute_accuracy(y_test, x_test, w_sgd, "SGD", threshold)
    print(f"F1 score SGD is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    accuracy, y_pred = implemented_functions.compute_accuracy(y_test, x_test, w_least_squares, "Least Squares", threshold)
    print(f"F1 score Least Squares is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    accuracy, y_pred = implemented_functions.compute_accuracy(y_test, x_test, best_ridge_weights, "Ridge", threshold)
    print(f"F1 score Ridge is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    accuracy, y_pred = implemented_functions.compute_accuracy(y_test, x_test, best_log_weights, "Logistic", threshold)
    print(f"F1 score Logistic is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    accuracy, y_pred = implemented_functions.compute_accuracy(y_test, x_test, best_log_reg_weights, "Regularized Logistic", threshold)
    print(f"F1 score Regularized Logistic is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    return y_pred

test_methods(x_test, y_test, w_sgd, w_least_squares, best_ridge_weights, best_log_weights, best_log_reg_weights, threshold=0.5)
#test_methods(x_test, y_test, w_sgd, w_least_squares, best_ridge_weights, best_log_weights, best_log_reg_weights, threshold=0.3)
#test_methods(x_test, y_test, w_sgd, w_least_squares, best_ridge_weights, best_log_weights, best_log_reg_weights, threshold=10)




Accuracy of SGD is 91.34651162790698%
F1 score SGD is 0.0
Accuracy of Least Squares is 91.24651162790698%
F1 score Least Squares is 0.15718763994626064
Accuracy of Ridge is 80.04883720930232%
F1 score Ridge is 0.3517944843218738
Accuracy of Logistic is 89.26046511627908%
F1 score Logistic is 0.37391540130151846
Accuracy of Regularized Logistic is 91.34651162790698%
F1 score Regularized Logistic is 0.0


array([0., 0., 0., ..., 0., 0., 0.])

In [8]:
w1 = np.sum(y_test == 1)
w1

832

In [ ]:
x_train = np.loadtxt('x_train_preprocessed.csv', delimiter=',')
x_test = np.loadtxt('x_test_preprocessed.csv', delimiter=',')
y_train = np.loadtxt('y_train_preprocessed.csv', delimiter=',')
y_pred = x_test@best_ridge_weights


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 575 is different from 561)

In [ ]:
best_ridge_weights.shape

(575,)

In [ ]:
x_train.shape

(8658, 575)

In [ ]:
x_test.shape

(50000, 561)